### 사용자 정의 데이터셋
- Pytorch에서 딥러닝 시 대량의 데이터 사용에 따른 부하(HW, SW) & 시간 소요에 대한 해결책으로 제시
- 대량 데이터셋 전용 처리 모듈 제공
- Dataset & DataLoader
	* DataSet => 데이터 전처리, 텐서화 등의 작업 진행
	* DataLoader => Dataset 인스턴스를 사용 & 배치크기만큼 데이터 추출


#### [1] 모듈 로딩 & 데이터 준비<hr>

In [4]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import pandas as pd
from sklearn.preprocessing import LabelEncoder			# 타겟 칼럼 수치화 (인코딩)

In [2]:
# 데이터 
DATA_FILE = r'C:\Users\KDP-43\Desktop\머신러닝_1\data\iris.csv'

In [3]:
irisDF = pd.read_csv(DATA_FILE)
irisDF.head(3)

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa


In [6]:
# 타겟 컬럼 수치화 ==> LabelEncoder
encoder = LabelEncoder()
encoder.fit(irisDF['variety'])
irisDF['variety'] = encoder.transform(irisDF['variety'])

print(irisDF.head(3))

   sepal.length  sepal.width  petal.length  petal.width  variety
0           5.1          3.5           1.4          0.2        0
1           4.9          3.0           1.4          0.2        0
2           4.7          3.2           1.3          0.2        0


#### [2] 사용자 정의 데이터셋 클래스 생성<hr>

In [11]:
# --------------------------------------------------
# 클래스 목적: 학습용 데이터셋 텐서화 & 전처리
# 클래스 이름: CustomDataSet
# 부모 클래스: torch.utils.data.DataSet
# 매개	 변수: featureDF, targetDF
# --------------------------------------------------

class CustomDataSet(Dataset):
    
	# 데이터 로딩 & 전처리 & 인스턴스 생성 메서드
    def __init__(self, featureDF, targetDF):
        super().__init__()
        self.featureDF = featureDF
        self.targetDF = targetDF
        self.n_rows = featureDF.shape[0]
        self.n_features = featureDF.shape[1]
    
	# 데이터 개수 반환 메서드
    def __len__(self):
        return self.n_rows
    
	# 특정 index 데이터 & 타겟 반환
    # Tensor 형태로
    def __getitem__(self, idx):
        featureTS = torch.FloatTensor( self.featureDF.iloc[idx].values )	# 시리즈 -> array -> tensor
        targetTS = torch.FloatTensor( self.targetDF.iloc[idx].values)
        return featureTS, targetTS
    


In [ ]:
# --------------------------------------------------
# 함수 목적: 파일 확장자별 DF 로드 & 변환
# 함수 이름: convertDataFrame
# 매개 변수: file_path
# 함수 결과: DataFrame
# --------------------------------------------------

def convertDataFrame(file_path, is_header=0):
    
	ext = file_path.rsplit('.')[-1]

	if ext == 'csv':
		return pd.read_csv(file_path, header=is_header)
	
	elif ext == 'json':
		return pd.read_json(file_path, header=is_header)
	
	elif ext in ['xlsx', 'xls']:
		return pd.read_excel(file_path, header=is_header)
	
	else:
		return pd.read_table(file_path, header=is_header )
	

    


In [ ]:
# --------------------------------------------------
# 클래스 목적: 파일 주소로 학습용 데이터셋 텐서화 & 전처리
# 클래스 이름: FileDataSet
# 부모 클래스: torch.utils.data.DataSet
# 매개	 변수: file_path
# --------------------------------------------------

class FileDataSet(Dataset):
    
	# 데이터 로딩 & 전처리 & 인스턴스 생성 메서드
    def __init__(self, file_path):
        super().__init__()

        dataDF = convertDataFrame(file_path)
        self.featuresDF = dataDF[dataDF.columns[:-1]]
        self.targetDF = dataDF[dataDF.columns[-1]]
        
        self.n_features = self.featuresDF.shape[1]
        self.n_rows = self.featuresDF.shape[0]
        
    
	# 데이터 개수 반환 메서드
    def __len__(self):
        return self.n_rows
    
	# 특정 index 데이터 & 타겟 반환
    # Tensor 형태로
    def __getitem__(self, idx):
        featureTS = torch.FloatTensor( self.featureDF.iloc[idx].values )	# 시리즈 -> array -> tensor
        targetTS = torch.FloatTensor( self.targetDF.iloc[idx].values)
        return featureTS, targetTS
    


#### [3] 데이터셋 인스턴스 생성 <hr>

In [16]:
featureDF, targetDF = irisDF[irisDF.columns[:-1]], irisDF[[irisDF.columns[-1]]]
print(f'featureDF => {featureDF.shape}, targetDF => {targetDF.shape}')

featureDF => (150, 4), targetDF => (150, 1)


In [17]:
# IRIS 데이터셋 인스턴스 생성
irisDS = CustomDataSet(featureDF, targetDF)

In [18]:
irisDS.featureDF

,sepal.length,sepal.width,petal.length,petal.width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


In [19]:
irisDS[0]

(tensor([5.1000, 3.5000, 1.4000, 0.2000]), tensor([0.]))

#### [4] 데이터로더 인스턴스 생성
- iterator 기능 있음

In [22]:
## 필요한 것: Dataset 인스턴스, Batch_size (default=1)
irisDL = DataLoader(irisDS, batch_size=5)

In [26]:
for dataTS, targetTS in irisDL:
    print("dataTS shape", dataTS.shape)
    print("targetTS shape", targetTS.shape)
    break

dataTS shape torch.Size([5, 4])
targetTS shape torch.Size([5, 1])
